In [1]:
import cv2
import torch
from ultralytics import YOLO

def detect_coke_bottle(image_path, model_path, real_height_cm=24):
    """
    Detects a  Coca-Cola bottle and calculates its real-world height.
    :param image_path: Path to the image.
    :param model_path: Path to the trained YOLO model.
    :param real_height_cm: Known real-world height of the  Coca-Cola bottle.
    :return: Bounding box height in pixels and estimated real-world height.
    """
    # Load the YOLO model
    model = YOLO(model_path)
    
    # Read the image
    image = cv2.imread(image_path)
    
    # Perform object detection
    results = model(image)
    
    # Get the detected bounding boxes
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Get bounding box coordinates
            height_pixels = y2 - y1  # Calculate height in pixels
            
            # Calculate Pixel-to-CM ratio
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            # print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
            # print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")
            
            return height_pixels, estimated_real_height
    
    print("No Coca-Cola bottle detected.")
    return None, None

# Example Usage
image_path = "Fine tune images/Nearly 3.5m from coke/image11.jpeg"  # Change to your image path
model_path = "best (8).pt"  # Path to your trained YOLO model
height_pixels, estimated_real_height =detect_coke_bottle(image_path, model_path)

print(f"Detected Coca-Cola Bottle Height: {height_pixels:.2f} pixels")
print(f"Estimated Real-World Height: {estimated_real_height:.2f} cm")


0: 800x608 1 750ml coke, 348.6ms
Speed: 4.3ms preprocess, 348.6ms inference, 1.0ms postprocess per image at shape (1, 3, 800, 608)
Detected Coca-Cola Bottle Height: 70.92 pixels
Estimated Real-World Height: 24.00 cm


In [2]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO

In [3]:
def load_model(model_path):
    model = YOLO(model_path)  # Load YOLO model
    return model

In [4]:
def detect_objects(model, image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Perform object detection
    return results, image


In [5]:


def get_ceiling_floor_coordinates(results):
    ceiling_y, floor_y = None, None
    
    for result in results:
        for box in result.boxes:
            cls = int(box.cls.cpu().numpy().item())  # Extract a single element safely
            y_min, y_max = int(box.xyxy[0][1]), int(box.xyxy[0][3])
            
            if cls == 0:  # Assuming class 0 is 'ceiling'
                ceiling_y = y_min
            elif cls == 1:  # Assuming class 1 is 'floor'
                floor_y = y_max
    
    return ceiling_y, floor_y

In [6]:
def calculate_height(ceiling_y, floor_y, calibration_obj=None):
    if ceiling_y is None or floor_y is None:
        print("Ceiling or Floor not detected!")
        return None
    
    pixel_height = floor_y - ceiling_y
    
    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        pixel_to_cm_ratio = real_height / obj_pixel_height
        real_ceiling_height = pixel_height * pixel_to_cm_ratio
        return real_ceiling_height
    
    return pixel_height  # Return only pixel height if no calibration object

In [7]:
# Example usage
model_path = "best (5).pt"  # Path to trained YOLO model
  # Input image path
calibration_obj = (estimated_real_height, height_pixels)  # Example: (real-world height in cm, pixel height)

model = load_model(model_path)
results, image = detect_objects(model, image_path)
ceiling_y, floor_y = get_ceiling_floor_coordinates(results)
pixel_height = floor_y - ceiling_y  # Calculate pixel height
print(f"Pixel Height: {pixel_height} pixels")  # Output the result

height = calculate_height(ceiling_y, floor_y, calibration_obj)
print(f"Ceiling Height: {height} cm" if calibration_obj else f"Ceiling Height: {height} pixels")



0: 640x480 3 ceilings, 6 floors, 3 wallss, 333.6ms
Speed: 2.8ms preprocess, 333.6ms inference, 17.2ms postprocess per image at shape (1, 3, 640, 480)
Pixel Height: 1275 pixels
Ceiling Height: 431.4547119140625 cm


In [8]:
import cv2

# Calculate pixel height
pixel_height = floor_y - ceiling_y

if ceiling_y is not None and floor_y is not None:
    # Draw a vertical line at the center of the image
    img_height, img_width, _ = image.shape
    center_x = img_width // 2  # Midpoint in width

    cv2.line(image, (center_x, int(ceiling_y)), (center_x, int(floor_y)), (0, 255, 0), 2)  # Green line

    # Save the image
    output_path = "output.jpg"
    cv2.imwrite(output_path, image)
    print(f"Image saved with vertical line at: {output_path}")

# Output pixel height
print(f"Pixel Height: {pixel_height} pixels")

# Calculate real-world height if calibration is provided
height = calculate_height(ceiling_y, floor_y, calibration_obj)
print(f"Ceiling Height: {height} cm" if calibration_obj else f"Ceiling Height: {height} pixels")


Image saved with vertical line at: output.jpg
Pixel Height: 1275 pixels
Ceiling Height: 431.4547119140625 cm
